In [1]:
import pandas as pd
import numpy as np
from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest
from hftbacktest import Recorder
from hftbacktest.stats import LinearAssetRecord

from src.strategies.glft_pretrained_rf import glft_pre_trained

In [2]:
MAKER_FEE = -0.0001
TAKER_FEE =  0.0005

gamma = 0.01143319447439636
delta = 1
adj1 = 0.9314120055506834
adj2 = 0.43081606500331926
max_position=50

stats = pd.read_csv("../data-generated/daily_stats_25_merged.csv").to_numpy()

In [3]:
day_start = 2


for i in range(0,4):
    print(f"--- WEEK {i} ---")
    day_end = day_start + 7
    data = []

    for day in range(day_start, day_end):
        day_file = f"../data/daily_processed/deribit_eth_perp_2025-01-{day:02d}.npz"
    
        data.append(day_file)
    
    eod = np.load(f"../data/snapshots/deribit_eth_perp_2025-01-{(day_start-1):02d}_eod.npz")['data']
    
    asset = (
    BacktestAsset()
        .data(data)
        .initial_snapshot(eod)
        .linear_asset(1.0)
        # .intp_order_latency(latencies, True)
        .constant_latency(10000, 10000) # Constant latency model (nanoseconds) values inspired from https://roq-trading.com/docs/blogs/2023-01-12/deribit/
        .risk_adverse_queue_model()
        # .power_prob_queue_model(2.0)
        .no_partial_fill_exchange()
        .trading_value_fee_model(MAKER_FEE, TAKER_FEE)
        .tick_size(0.01) # Tick size of this asset: minimum price increasement
        .lot_size(1) # Lot size of this asset: minimum trading unit
        # .roi_lb(0.0) # Sets the lower bound price for the range of interest in the market depth.
        # .roi_ub(3000.0) # Sets the upper bound price for the range of interest in the market depth.
        .last_trades_capacity(10000)
    )
    
    hbt = HashMapMarketDepthBacktest([asset])
    
    n_trading_days = day_end - day_start
    recorder = Recorder(1, n_trading_days* 1_000_000)
    
    execution_times, tree_train_times = glft_pre_trained(hbt, recorder.recorder, n_trading_days, gamma, delta, adj1, adj2, max_position, stats, day_start)
    hbt.close()
    
    stats = LinearAssetRecord(recorder.get(0)).stats()
    np.save(f"../results/rf/2-8Jan4/stats_week_{i+1}.npy",stats.summary())
    
    recorder.to_npz(f"../results/rf/2-8Jan4/recorder_week_{i+1}.npz")
    
    np.save(f"../results/rf/2-8Jan4/execution_times_week_{i+1}.npy", execution_times)
    np.save(f"../results/rf/2-8Jan4/tree_train_times_week_{i+1}.npy", tree_train_times)
    
    day_start = day_end

--- WEEK 0 ---
Hour: 0
Hour: 1
Hour: 2
Hour: 3
Hour: 4
Hour: 5
Hour: 6
Hour: 7
Hour: 8
Hour: 9
Hour: 10
Hour: 11
Hour: 12
Hour: 13
Hour: 14
Hour: 15
Hour: 16
Hour: 17
Hour: 18
Hour: 19
Hour: 20
Hour: 21
Hour: 22
Hour: 23
Hour: 0
Hour: 1
Hour: 2
Hour: 3
Hour: 4
Hour: 5
Hour: 6
Hour: 7
Hour: 8
Hour: 9
Hour: 10
Hour: 11
Hour: 12
Hour: 13
Hour: 14
Hour: 15
Hour: 16
Hour: 17
Hour: 18
Hour: 19
Hour: 20
Hour: 21
Hour: 22
Hour: 23
Hour: 0
Hour: 1
Hour: 2
Hour: 3
Hour: 4
Hour: 5
Hour: 6
Hour: 7
Hour: 8
Hour: 9
Hour: 10
Hour: 11
Hour: 12
Hour: 13
Hour: 14
Hour: 15
Hour: 16
Hour: 17
Hour: 18
Hour: 19
Hour: 20
Hour: 21
Hour: 22
Hour: 23
Hour: 0
Hour: 1
Hour: 2
Hour: 3
Hour: 4
Hour: 5
Hour: 6
Hour: 7
Hour: 8
Hour: 9
Hour: 10
Hour: 11
Hour: 12
Hour: 13
Hour: 14
Hour: 15
Hour: 16
Hour: 17
Hour: 18
Hour: 19
Hour: 20
Hour: 21
Hour: 22
Hour: 23
Hour: 0
Hour: 1
Hour: 2
Hour: 3
Hour: 4
Hour: 5
Hour: 6
Hour: 7
Hour: 8
Hour: 9
Hour: 10
Hour: 11
Hour: 12
Hour: 13
Hour: 14
Hour: 15
Hour: 16
Hour: 17
Hour: 18


TypingError: Failed in nopython mode pipeline (step: Handle with contexts)
[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at D:\Repositories\BIM-Master-Thesis\src\strategies\glft_pretrained_rf.py (97)[0m
[1m
File "strategies\glft_pretrained_rf.py", line 97:[0m
[1mdef get_current_day(current_timestamp):
    <source elided>

[1m@njit
[0m[1m^[0m[0m

[0m[1mDuring: Pass nopython_type_inference[0m
[0m[1mDuring: Pass with_lifting[0m 

This error may have been caused by the following argument(s):
- argument 8: [1mCannot determine Numba type of <class 'hftbacktest.stats.stats.Stats'>[0m
